## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-08-20-53-27 +0000,bbc,Three-year-old girl killed in Gaza by Israeli ...,https://www.bbc.com/news/articles/cqxqz4227x7o...
1,2025-12-08-20-49-00 +0000,wsj,Europe Scrambles for Influence in U.S.-Led Pea...,https://www.wsj.com/world/europe/zelensky-rall...
2,2025-12-08-20-49-00 +0000,wsj,Opinion | World Trade Grows Without the U.S.,https://www.wsj.com/opinion/world-trade-grows-...
3,2025-12-08-20-49-00 +0000,wsj,U.S. Treasury Yields End Higher,https://www.wsj.com/finance/ten-year-jgb-yield...
4,2025-12-08-20-45-00 +0000,wsj,U.S. Plans to Approve Export of Nvidia’s H200 ...,https://www.wsj.com/tech/nvidia-china-exports-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
36,trump,48
118,warner,18
63,new,16
256,court,14
238,deal,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
149,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...,150
164,2025-12-08-14-53-40 +0000,latimes,Paramount launches hostile $78-billion bid for...,https://www.latimes.com/business/story/2025-12...,127
174,2025-12-08-14-06-59 +0000,nyt,Trump Looms Over Netflix’s Deal for Warner Bro...,https://www.nytimes.com/2025/12/08/business/de...,113
206,2025-12-08-10-44-44 +0000,bbc,Trump says $72bn Netflix-Warner Bros deal 'cou...,https://www.bbc.com/news/articles/cn815egjqjpo...,113
256,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
149,150,2025-12-08-15-28-04 +0000,nypost,Paramount Skydance launches hostile bid for Wa...,https://nypost.com/2025/12/08/business/paramou...
37,59,2025-12-08-19-41-15 +0000,bbc,Alina Habba resigns after court disqualified h...,https://www.bbc.com/news/articles/cj69x814dkyo...
44,48,2025-12-08-19-34-06 +0000,nypost,‘Devil’ son who pushed 64-year-old mom out of ...,https://nypost.com/2025/12/08/us-news/devil-so...
52,45,2025-12-08-19-23-51 +0000,nypost,China’s trade surplus hits $1 trillion for fir...,https://nypost.com/2025/12/08/business/chinas-...
182,43,2025-12-08-13-20-35 +0000,nypost,‘Disappointed’ Trump claims Zelensky ‘hasn’t y...,https://nypost.com/2025/12/08/world-news/disap...
67,43,2025-12-08-18-44-00 +0000,wsj,Supreme Court Appears Poised to Give President...,https://www.wsj.com/politics/policy/supreme-co...
77,38,2025-12-08-18-21-58 +0000,nyt,Trump Plans $12 Billion Bailout to Aid Farmers...,https://www.nytimes.com/2025/12/08/us/politics...
86,36,2025-12-08-18-00-42 +0000,nypost,Students form human swastika on Calif. high sc...,https://nypost.com/2025/12/08/us-news/human-sw...
112,35,2025-12-08-17-05-06 +0000,nypost,"Berkshire Hathaway’s Todd Combs, also CEO of G...",https://nypost.com/2025/12/08/business/berkshi...
162,35,2025-12-08-14-56-20 +0000,nypost,7.2-magnitude earthquake strikes off Japan’s c...,https://nypost.com/2025/12/08/world-news/7-2-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
